# PyTorch를 사용한 경사하강법 및 선형 회귀

## 이 노트북에서 다룰 내용
1. 선형 회귀 및 경사하강법 소개
2. PyTorch 텐서를 사용하여 선형 회귀 모델 구현
3. 경사하강법을 이용한 선형 회귀 모델 학습
4. 내장된 PyTorch를 사용하여 경사하강법 및 선형 회귀 구현

시작하기 전에 필요한 라이브러리를 설치해야 한다.

PyTorch의 설치는 운영 체제/클라우드 환경에 따라 다를 수 있다. 
자세한 설치 지침은 https://pytorch.org에서 찾을 수 있습니다.

In [2]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

## 선형 회귀 소개

이 노트북에서는 기계 학습의 기본 알고리즘 중 하나인 *선형 회귀*에 대해 설명한다.

지역의 평균 기온, 강우량, 습도(*입력 변수 또는 특징*)를 보고 사과와 오렌지(*목표 변수*)의 작물 수확량을 예측하는 모델을 만들 것이다. 훈련 데이터는 아래와 같다.


![선형-회귀-훈련-데이터](https://i.imgur.com/6Ujttb4.png)

선형 회귀 모델에서 각 목표 변수는 입력 변수의 가중합(weighted sum)으로 추정되며 편향(bias)이라고 하는 상수가 추가된다.

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

시각적으로 사과의 수확량은 온도, 강우량 및 습도의 선형 또는 평면 함수임을 의미한다.

![선형 회귀 그래프](https://i.imgur.com/4DJ9f8X.png)

선형 회귀의 *학습* 부분은 새로운 데이터에 대한 정확한 예측을 하기 위해 훈련 데이터를 사용하여 적절한 가중치 'w11, w12,... w23, b1 & b2'값의 집합을 찾아내는 것이다.

_학습된_ 가중치는 해당 지역의 평균 온도, 강우량 및 습도를 사용하여 새 지역에서 사과와 오렌지의 수확량을 예측하는 데 사용된다.



우리는 *경사하강법*이라는 최적화 기술을 사용하여 더 나은 예측을 위해 가중치를 약간 여러 번 조정하여 모델을 _훈련_시킬 것이다.

Numpy와 PyTorch를 가져오는 것부터 시작하자.

In [3]:
import numpy as np
import torch

## 훈련 데이터

우리는 '입력'과 '목표'라는 두 개의 행렬을 사용하여 훈련 데이터를 나타낼 수 있으며, 각각 관찰당 하나의 행과 변수당 하나의 열이 있다.

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

입력 변수와 대상 변수는 별도로 작업할 것이기 때문에 분리했다.

또한 일반적으로 훈련 데이터로 작업하는 방식이기 때문에 numpy 배열을 만들었다.

일부 CSV 파일을 numpy 배열로 읽고 일부 처리를 수행한 다음 PyTorch 텐서로 변환한다.

배열을 PyTorch 텐서로 변환해 보자.

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## 선형 회귀 모델

가중치와 편향(`w11, w12,... w23, b1 & b2`)은 임의의 값으로 초기화되는 행렬로 나타낼 수도 있다.

`w`의 첫 번째 행과 `b`의 첫 번째 요소는 첫 번째 대상 변수, 즉 사과의 수확량을 예측하는 데 사용되며 유사하게 두 번째는 오렌지 일때도 사용된다.

In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.0799,  1.5909, -1.4551],
        [-0.5223,  0.9004,  1.1792]], requires_grad=True)
tensor([-0.1269, -0.0251], requires_grad=True)


`torch.randn`은 평균이 0이고 표준편차가 1인 [정규 분포](https://en.wikipedia.org/wiki/Normal_distribution)에서 무작위로 선택된 요소를 사용하여 주어진 모양으로 텐서를 생성한다.

우리의 *모델*은 단순히 `입력`과 가중치 `w`(전치)의 행렬 곱셈을 수행하고 편향 `b`(각 관찰에 대해 복제됨)을 추가하는 함수다.

![매트릭스 다중](https://i.imgur.com/WGXLFvA.png)

다음과 같이 모델을 정의할 수 있다.

In [8]:
def model(x):
    return x @ w.t() + b

`@`는 PyTorch에서 행렬 곱셈을 나타내고 `.t` 메서드는 텐서의 전치를 반환한다.

입력 데이터를 모델에 전달하여 얻은 행렬은 대상 변수에 대한 예측 집합이다.

In [9]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 38.0630,  72.8767],
        [ 39.4773, 107.1464],
        [121.7068, 143.5788],
        [  6.2963,  29.0438],
        [ 45.2311, 132.9165]], grad_fn=<AddBackward0>)


우리 모델의 예측을 실제 목표와 비교해 봅시다.

In [10]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


무작위 가중치와 편향으로 모델을 초기화했기 때문에 모델의 예측과 실제 목표 사이에 큰 차이가 있음을 알 수 있을 것이다.

분명히, 무작위로 초기화된 모델이 *잘 작동*할 것이라고 기대할 수는 없을 것이다.

## 손실 함수

모델을 개선하기 전에 모델이 얼마나 잘 수행되고 있는지 평가할 방법이 필요하다. 다음 방법을 사용하여 모델의 예측을 실제 목표와 비교할 수 있다.

* 두 행렬(`preds` 및 `targets`) 간의 차이를 계산한다.

* 음수 값을 제거하기 위해 차분 행렬의 모든 요소를 제곱한다.

* 결과 행렬의 요소 평균을 계산한다.

결과는 **평균 제곱 오차**(MSE)로 알려진 단일 숫자이다.

In [11]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum`은 텐서에 있는 모든 요소의 합을 반환한다.

텐서의 `.numel` 메서드는 텐서의 요소 수를 반환한다.

우리 모델의 현재 예측에 대한 평균 제곱 오차를 계산해 보자.

In [12]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(605.1987, grad_fn=<DivBackward0>)


결과를 해석하면, *평균적으로 예측의 각 요소는 손실의 제곱근만큼 실제 목표와 다르다*.

그리고 우리가 예측하려는 숫자 자체가 50-200 범위에 있다는 점을 고려할 때 이는 매우 나쁘다.

결과는 모델이 목표 변수를 예측하는 데 얼마나 나쁜지를 나타내기 때문에 *손실*이라고 말한다.

이는 모델의 정보 손실을 나타낸다. 손실이 낮을수록 더 좋은 모델임을 나타낸다.

## 그라디언트 계산

PyTorch를 사용하면 손실 w.r.t의 기울기 또는 도함수를 자동으로 계산할 수 있다.

`requires_grad`가 `True`로 설정되어 있기 때문에 가중치와 편향을 사용할 수 있다.

In [13]:
# Compute gradients
loss.backward()

기울기는 각 텐서의 `.grad` 속성에 저장된다.

손실 w.r.t의 미분에 유의하라. 가중치 행렬은 그 자체가 같은 차원의 행렬이다.

In [14]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.0799,  1.5909, -1.4551],
        [-0.5223,  0.9004,  1.1792]], requires_grad=True)
tensor([[-2088.0627, -2142.8301, -1579.3232],
        [  367.6758,   629.0081,   362.0818]])


## 손실을 줄이기 위해 가중치와 편향을 조정합니다.

손실은 가중치와 편향의 [2차 함수](https://en.wikipedia.org/wiki/Quadratic_function)이며, 우리의 목표는 손실이 가장 낮은 가중치 집합을 찾는 것이다.

개별 가중치 또는 편향 요소에 대한 손실 그래프를 플롯하면 아래 그림과 같이 표시된다.

미적분학의 중요한 통찰력은 기울기가 손실의 변화율, 즉 손실 함수의 [slope](https://en.wikipedia.org/wiki/Slope) w.r.t를 나타낸다.

그래디언트 요소가 **양수**인 경우:

* **가중치 요소의 값을 약간 증가**하면 손실이 **증가**
* **가중치 요소의 값을 약간 감소**하면 손실이 **감소**

![사위 그라데이션](https://i.imgur.com/WLzJ4xP.png)

그래디언트 요소가 **음수**인 경우:

* **가중치 요소의 값을 약간 증가**하면 손실이 **감소**
* **가중치 요소의 값을 약간 낮추면** 손실이 **증가**

![음수=그라디언트](https://i.imgur.com/dvG2fxU.png)

가중치 요소를 변경하여 손실의 증가 또는 감소는 손실 w.r.t의 기울기에 비례한다. 
이러한 내용은 _gradient_를 따라 _descending_ 모델을 개선하는 데 사용할 _gradient descent_ 최적화 알고리즘의 기초이다.

손실 w.r.t의 도함수에 비례하는 소량을 각 가중치 요소에서 뺄 수 있다. 이 요소로 손실을 약간 줄이는 것입니다.

In [15]:
w
w.grad

tensor([[-2088.0627, -2142.8301, -1579.3232],
        [  367.6758,   629.0081,   362.0818]])

In [16]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

가중치를 너무 많이 수정하지 않도록 매우 작은 수(이 경우 '10^-5')로 그래디언트에 곱한다.

이로써, 우리는 내리막 길로 작은 발걸음을 내디뎠다고 볼 수 있다.

이 수치를 알고리즘의 *학습률*이라고 한다.

우리는 가중치와 편향을 업데이트하는 동안 그래디언트를 추적, 계산 또는 수정해서는 안 된다고 PyTorch에 나타내기 위해 `torch.no_grad`를 사용한다.

In [17]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(605.1987, grad_fn=<DivBackward0>)


계속 진행하기 전에 `.zero_()` 메서드를 호출하여 그래디언트를 0으로 재설정해야한다.

PyTorch가 그라디언트를 축적하기 때문에 이 작업을 수행해야한다.

그렇지 않으면 다음에 손실에 대해 `.backward`를 호출할 때 새 그라디언트 값이 기존 그라디언트에 추가되어 예기치 않은 결과가 발생할 수 있다.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## 경사하강법을 사용한 모델 훈련

이제 우리는 경사하강 최적화 알고리즘을 사용하여 손실을 줄이고 모델을 개선할 수 있다. 
따라서 다음 단계를 사용하여 모델을 _학습(train)_할 수 있을 것이다.

1. 예측 생성
2. 손실 계산
3. 가중치와 편향으로 기울기 계산
4. 기울기에 비례하는 소량을 빼서 가중치 조정
5. 그라디언트를 0으로 재설정

위의 단계를 단계별로 구현해 보자.

In [19]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 41.7023,  72.0311],
        [ 44.2742, 106.0265],
        [127.3110, 142.2060],
        [  9.9322,  28.2643],
        [ 49.8347, 131.8054]], grad_fn=<AddBackward0>)


In [20]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(494.8894, grad_fn=<DivBackward0>)


In [21]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-1712.4089, -1739.7845, -1330.2585],
        [  279.8259,   533.1392,   303.2268]])
tensor([-21.5891,   4.0666])


위에서 계산된 그래디언트를 사용하여 가중치와 편향을 업데이트해 보자.

In [22]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [23]:
print(w)
print(b)

tensor([[-0.0419,  1.6297, -1.4260],
        [-0.5288,  0.8888,  1.1726]], requires_grad=True)
tensor([-0.1264, -0.0252], requires_grad=True)


새로운 가중치와 편향을 사용하면 모델의 손실이 낮을 것이다.

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(420.3148, grad_fn=<DivBackward0>)


기울기 하강법을 사용하여 가중치와 편향을 약간 조정하는 것만으로 손실을 줄일 수 있다는 것을 확인할 수 있을 것이다.

## 반복을 통한 훈련

손실을 더 줄이기 위해 기울기를 사용하여 가중치와 편향을 조정하는 과정을 여러 번 반복할 수 있다.

각 반복을 _epoch_라고 한다.  이제 100 Epoch 동안 모델을 훈련시켜보자.

In [26]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

다시 한 번 손실이 더 낮아졌는지 확인해보자.

In [27]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(170.9299, grad_fn=<DivBackward0>)


손실은 이제 초기 값보다 훨씬 낮다. 모델의 예측을 보고 타겟(실제 값)과 비교해 보자.

In [28]:
# Predictions
preds

tensor([[ 58.5064,  69.5755],
        [ 69.1907, 101.6759],
        [146.1393, 131.8502],
        [ 27.5950,  32.9276],
        [ 75.4510, 123.2680]], grad_fn=<AddBackward0>)

In [29]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

이제 예측이 목표 변수(타겟)에 매우 가깝습니다.

몇 epoch를 더 훈련하면 더 나은 결과를 얻을 수 있다.

## PyTorch 내장을 사용한 선형 회귀

우리는 앞에서 몇 가지 기본 텐서 연산을 사용하여 선형 회귀 및 경사 하강 모델을 구현했다.

그러나 이것은 딥 러닝의 일반적인 패턴이기 때문에 PyTorch는 몇 줄의 코드로 모델을 쉽게 만들고 훈련할 수 있도록 여러 내장 함수와 클래스를 제공한다.

신경망 구축을 위한 유틸리티 클래스가 포함된 PyTorch에서 `torch.nn` 패키지를 가져오는 것으로 시작한다.

In [30]:
import torch.nn as nn

이전과 마찬가지로 입력, 목표 및 행렬을 나타낸다.

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [32]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## 데이터세트와 데이터로더

우리는 `입력(inputs)`과 `출력, 목표변수(targets)`의 행에 튜플로 액세스할 수 있는 `TensorDataset`을 생성할 수 있다.
PyTorch에서 다양한 유형의 데이터 세트 작업을 위한 표준 API를 제공한다.

In [34]:
from torch.utils.data import TensorDataset

In [35]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

`TensorDataset`을 사용하면 배열 인덱싱 표기법(위 코드의 '[0:3]')을 사용하여 훈련 데이터의 작은 섹션에 액세스할 수 있다.

두 개의 요소가 있는 튜플을 반환한다.

첫 번째 요소에는 선택한 행에 대한 입력 변수가 포함되고 두 번째 요소에는 대상이 포함된다.

또한 훈련하는 동안 데이터를 미리 정의된 크기의 배치로 분할할 수 있는 `DataLoader`를 만들 것이다. 

`DataLoader`는 데이터 셔플링 및 무작위 샘플링과 같은 기타 유틸리티도 제공한다.

In [36]:
from torch.utils.data import DataLoader

In [37]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

우리는 `for` 루프에서 데이터 로더를 사용할 수 있다.

In [38]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[101.,  44.,  37.],
        [ 88., 134.,  59.],
        [ 68.,  97.,  70.],
        [ 92.,  87.,  64.],
        [102.,  43.,  37.]])
tensor([[ 21.,  38.],
        [118., 132.],
        [102., 120.],
        [ 82., 100.],
        [ 22.,  37.]])


각 반복에서 데이터 로더는 지정된 배치 크기의 데이터 배치 하나를 반환한다.

`shuffle`이 `True`로 설정되어 있으면 배치를 생성하기 전에 훈련 데이터를 섞을 수 있다.

셔플링은 최적화 알고리즘에 대한 입력을 무작위화하여 손실을 더 빠르게 줄이는 데 도움이 된다.

## nn.Linear

가중치와 편향을 수동으로 초기화하는 대신 PyTorch의 `nn.Linear` 클래스를 사용하여 모델을 정의할 수 있다.

In [39]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[0.3217, 0.2614, 0.1505],
        [0.1617, 0.1871, 0.3047]], requires_grad=True)
Parameter containing:
tensor([-0.5377,  0.0781], requires_grad=True)


PyTorch 모델에는 모델에 있는 모든 가중치와 편향 행렬을 포함하는 리스트를 반환하는 유용한 `.parameters` 메서드가 있다.

선형 회귀 모델의 경우 하나의 가중치 행렬과 하나의 편향 행렬을 가지고 있다.

In [40]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[0.3217, 0.2614, 0.1505],
         [0.1617, 0.1871, 0.3047]], requires_grad=True),
 Parameter containing:
 tensor([-0.5377,  0.0781], requires_grad=True)]

모델을 사용하여 이전과 같은 방식으로 예측을 생성할 수 있다.

In [45]:
# Generate predictions
preds = model(inputs)
preds

tensor([[46.9318, 37.5157],
        [61.3721, 50.7525],
        [71.2055, 56.8847],
        [49.0864, 35.8856],
        [57.2877, 50.5206],
        [46.9922, 37.4902],
        [61.2612, 50.8700],
        [71.6777, 57.3510],
        [49.0260, 35.9110],
        [57.1164, 50.6636],
        [46.8209, 37.6332],
        [61.4324, 50.7271],
        [71.3164, 56.7672],
        [49.2576, 35.7426],
        [57.2273, 50.5460]], grad_fn=<AddmmBackward>)

## 손실 함수

손실 함수를 수동으로 정의하는 대신 내장 손실 함수 `mse_loss`를 사용할 수 있다.

In [46]:
# Import nn.functional
import torch.nn.functional as F

`nn.functional` 패키지에는 많은 유용한 손실 기능과 기타 여러 유틸리티가 포함되어 있다.

In [47]:
# Define loss function
loss_fn = F.mse_loss

우리 모델의 현재 예측에 대한 손실을 계산해 보자.

In [48]:
loss = loss_fn(preds, targets)
print(loss)

tensor(1957.0083, grad_fn=<MseLossBackward>)


## 최적화

그라디언트를 사용하여 모델의 가중치 및 편향을 수동으로 조작하는 대신 최적화 `optim.SGD`를 사용할 수 있다.

SGD는 `"stochastic gradient descent"`의 약자이다.

_stochastic_이라는 용어는 샘플이 단일 그룹 대신 무작위 배치로 선택됨을 나타낸다.

In [49]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

`model.parameters()`는 `optim.SGD`에 인수로 전달되어 옵티마이저가 업데이트 단계에서 어떤 행렬을 수정해야 하는지 알 수 있다.

또한 매개변수가 수정되는 양을 제어하는 학습률을 지정할 수 있다.

## 모델 훈련

이제 모델을 훈련할 준비가 되었다. 우리는 경사 하강법을 구현하기 위해 동일한 프로세스를 따를 것이다.

1. 예측 생성

2. 손실 계산

3. 가중치와 편향으로 기울기 계산

4. 기울기에 비례하는 소량을 빼서 가중치를 조정

5. 그라디언트를 0으로 재설정

유일한 변경 사항은 모든 반복에서 전체 교육 데이터를 처리하는 대신 배치단위의 데이터를 처리한다는 것이다.

주어진 epoch 수에 대해 모델을 훈련시키는 유틸리티 함수 `fit`을 정의해 보겠다.

In [50]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

위에서 주의해야 할 사항:

* 우리는 이전에 정의한 데이터 로더를 사용하여 모든 반복에 대한 데이터 배치를 가져온다.

* 매개변수(가중치 및 편향)를 수동으로 업데이트하는 대신 `opt.step`을 사용하여 업데이트를 수행하고 `opt.zero_grad`를 사용하여 그래디언트를 0으로 재설정한다.

* 훈련 진행 상황을 추적하기 위해 매 10번째 epoch에 대한 마지막 데이터 배치의 손실을 인쇄하는 로그 문도 추가했다. `loss.item`은 손실 텐서에 저장된 실제 값을 반환한다.

100 Epoch 동안 모델을 훈련시키자.

In [51]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 331.0684
Epoch [20/100], Loss: 423.3690
Epoch [30/100], Loss: 294.8078
Epoch [40/100], Loss: 98.8626
Epoch [50/100], Loss: 88.5521
Epoch [60/100], Loss: 43.0958
Epoch [70/100], Loss: 50.0133
Epoch [80/100], Loss: 15.1282
Epoch [90/100], Loss: 35.0053
Epoch [100/100], Loss: 19.7186


모델을 사용하여 예측을 생성하고 목표에 가까운지 확인해보자.

In [52]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.1850,  71.5359],
        [ 80.6428,  99.1659],
        [119.2262, 134.1587],
        [ 28.4053,  44.0335],
        [ 94.9190, 112.4586],
        [ 57.0892,  70.5703],
        [ 80.1648,  98.9442],
        [119.3931, 134.6269],
        [ 29.5011,  44.9991],
        [ 95.5367, 113.2026],
        [ 57.7069,  71.3143],
        [ 79.5471,  98.2002],
        [119.7043, 134.3803],
        [ 27.7876,  43.2895],
        [ 96.0148, 113.4242]], grad_fn=<AddmmBackward>)

In [53]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

실제로, 예측은 우리의 목표와 근접한 값을 갖는다.

이제, 우리는 한 지역의 평균 기온, 강우량, 습도를 보고 사과와 오렌지의 수확량을 예측할 수 있도록 합리적으로 좋은 모델을 훈련했다고 말할 수 있을 것이다.

단일 입력 행을 포함하는 배치를 전달하여 새 지역의 작물 수확량을 예측하는 데 사용할 수 있을 것이다.

In [54]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.6451, 68.5552]], grad_fn=<AddmmBackward>)

사과의 예상 수확량은 헥타르당 54.3톤이고 오렌지는 헥타르당 68.3톤이다.

## 기계 학습 vs. 명시적 프로그래밍

이 노트북에서 우리가 취한 접근 방식은 여러분이 알고 있는 프로그래밍과 매우 다르다.일반적으로 우리는 일부 입력을 받고 일부 작업을 수행하고 결과를 반환하는 프로그램을 작성한다.

그러나 이 노트북에서는 일부 알려지지 않은 매개변수(가중치 및 편향)를 사용하여 표현되는 입력과 출력 간의 특정 관계를 가정하는 `"모델"`을 정의했다. 그런 다음 모델에 일부 알려진 입력 및 출력을 보여주고 모델을 _train_하여 알려지지 않은 매개변수에 대한 좋은 값을 제시했다. 훈련을 마치면 모델을 사용하여 새 입력에 대한 출력을 계산할 수 있다.


이러한 프로그래밍 패러다임은 _기계 학습(Machine Learning)_으로 알려져 있으며, 여기서 데이터를 사용하여 입력과 출력 간의 관계를 파악하는 방식으로 동작한다. _딥 러닝(Deep Learning)_은 행렬 연산, 비선형 활성화 함수 및 경사하강법을 사용하여 모델을 구축하고 훈련하는 기계 학습의 한 분야이다.

Tesla Motors의 AI 이사인 Andrej Karpathy는 [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35)이라는 제목으로 이 주제에 대한 훌륭한 블로그 게시물이 있다.

Francois Chollet의 책 [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)의 이 그림은 명시적 프로그래밍과 기계 학습의 차이점에 초점을 두었다.

![](https://i.imgur.com/oJEQe7k.png)
